In [1]:
#1. Import packages
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import os, datetime

#2. Load the csv data
base_path = r"/content/drive/MyDrive/Heart_disease"
heart_path = os.path.join(base_path, 'heart.csv')

heart_data = pd.read_csv(heart_path)

In [2]:
#%%
#3. Data preparation
print(heart_data.isna().sum())

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64


In [3]:
#%%
#(a) Drop unwanted features
heart_data = heart_data.drop(['age', 'oldpeak'], axis=1)

In [4]:
#%%
#(b) One hot encode the categorical features
heart_data = pd.get_dummies(data=heart_data)

In [5]:
#%%
#(c) Impute data to fill up missing values
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
heart_data_imputed = imputer.fit_transform(heart_data)

In [7]:
#%%
#(d) extract out the training labels
train_features = heart_data_imputed[:,1:]
train_labels = heart_data_imputed[:,0]

In [8]:
#%%
#(e) Perform train test split on train data (for train-validation split)
SEED = 12345
x_train, x_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.2, random_state=SEED)

In [9]:
#%%
#(f) Data normalization
standardizer = StandardScaler()
standardizer.fit(x_train)
x_train = standardizer.transform(x_train)
x_test = standardizer.transform(x_test)

In [16]:
#%%
#4. Model creation
nClass = len(np.unique(y_test))
nIn = x_train.shape[1]

#Use funtional API to build NN
inputs = keras.Input(shape=(nIn,))
h1 = layers.Dense(128, activation= 'relu')
h2 = layers.Dropout(0.5)
h3 = layers.Dense(64, activation = 'relu')
h4 = layers.Dropout(0.5)
h5 = layers.Dense(32, activation = 'relu')
out_layer = layers.Dense(nClass, activation = 'softmax')

#Chain the layers with funtional API
x = h1(inputs)
x = h2(x)
x = h3(x)
x = h4(x)
x = h5(x)
outputs = out_layer(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='heart_disease_model')
model.summary()

Model: "heart_disease_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 11)]              0         
                                                                 
 dense_4 (Dense)             (None, 128)               1536      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 2)         

In [17]:
#%%
#Compile the model
model.compile(optimizer='adam', loss ='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
#%%
#Early stopping callback
base_log_path = r"C:\Users\FurqanFaiz Ishak\Desktop\AI and Machine Learning Competence for Industry 4.0\DeepLearning\Exercises\log"
log_path = os.path.join(base_log_path, 'project_1', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tb = TensorBoard(log_dir=log_path)
es = EarlyStopping(patience=10, verbose=1,restore_best_weights=True)

In [19]:
#%%
BATCH_SIZE = 32
EPOCHS = 200
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[es, tb])

Epoch 1/200
26/26 [==============================] - 1s 10ms/step - loss: 0.6106 - accuracy: 0.6915 - val_loss: 0.5986 - val_accuracy: 0.6927
Epoch 2/200
26/26 [==============================] - 0s 4ms/step - loss: 0.5926 - accuracy: 0.7000 - val_loss: 0.5646 - val_accuracy: 0.7073
Epoch 3/200
26/26 [==============================] - 0s 4ms/step - loss: 0.5574 - accuracy: 0.7146 - val_loss: 0.5339 - val_accuracy: 0.7415
Epoch 4/200
26/26 [==============================] - 0s 4ms/step - loss: 0.5374 - accuracy: 0.7049 - val_loss: 0.5098 - val_accuracy: 0.7512
Epoch 5/200
26/26 [==============================] - 0s 4ms/step - loss: 0.5224 - accuracy: 0.7268 - val_loss: 0.4850 - val_accuracy: 0.7463
Epoch 6/200
26/26 [==============================] - 0s 4ms/step - loss: 0.4955 - accuracy: 0.7476 - val_loss: 0.4646 - val_accuracy: 0.7561
Epoch 7/200
26/26 [==============================] - 0s 4ms/step - loss: 0.5005 - accuracy: 0.7402 - val_loss: 0.4487 - val_accuracy: 0.7463
Epoch 8/200
